<a href="https://colab.research.google.com/github/nicks165/VectorDatabases/blob/main/Weaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://weaviate.io/developers/weaviate/quickstart/

In [1]:
!pip install weaviate-client cohere datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.

Initiatize Weaviate

In [2]:
import weaviate

# Instantiate the client with the auth config
client = weaviate.Client(
    url="https://cohere-wikipedia-td1xy2jz.weaviate.network",  # Replace w/ your endpoint
    #auth_client_secret=weaviate.AuthClientPassword(
    #    username = "nicks165@gmail.com",  # Replace w/ your WCS username
    #    password = "12qwaszxQW",   # Replace w/ your Weaviate password
    #)
)

Load datset and get the embeddings from Cohere dataset at Huggingface

In [ ]:
from datasets import load_dataset
import torch
import cohere

#Load all documents + embeddings
limit = -1 # keep -1 for all, else update to a positive number to limit

max_docs_loaded = 0
docs_stream = load_dataset(f"Cohere/wikipedia-22-12-zh-embeddings", split="train", streaming=True)
docs = []
doc_embeddings = []

for doc in docs_stream:
    docs.append(doc)
    doc_embeddings.append(doc['emb'])
    max_docs_loaded += 1
    if (limit > 0 and max_docs_loaded == limit):
      break

Initialize index/collection (class in weaviate)

In [ ]:
# Class definition object. Weaviate's autoschema feature will infer properties when importing.
class_obj = {
    "class": "wikipedia_articles",
    "vectorizer": "none",
}

# Add the class to the schema
client.schema.create_class(class_obj)

Confirm shape and save the dataset to disk

In [ ]:
import numpy as np

shape = np.array(doc_embeddings).shape
print(shape)

#docs_stream.save_to_disk("cohere dataset")

Load embeddings and metadata into Weaviate

In [ ]:
import time

# Configure a batch process
with client.batch as batch:
  batch.batch_size=100
   

  # start timer
  start_time = time.time()

  # Batch import all Questions
  for i, doc in enumerate(docs):
    
    # create metadata dictionary
    properties = {
            "text": doc["text"],
            }

    client.batch.add_data_object(properties, class_obj["class"], vector=doc_embeddings[i])

    # i+1 because index starts at 0 :)
    if(i+1 in [shape[0], batch.batch_size, 1000, 10000, 100000, 500000, 1000000, 2000000]):
      print(" For {0} entries, time taken for inserts = {1} ".format(i+1, time.time() - start_time))


# let's view the index statistics
print("==========================================")
print(client.query.aggregate(class_obj["class"]).with_meta_count().do())

Querying the dataset

In [ ]:
import json

co = cohere.Client(f"o7lTEJeC1QHjU5I4Ee6U2I0m6l5wCOUPWqwoGM7H")  # Add your cohere API key from www.cohere.com

query1 = "What was the cause of the major recession in the early 20th century?"
query2 = "Where is Mount Everest?"
query3 = ""

# create the query embedding
nearVector = {
    "vector": co.embed(texts=[query1], model='multilingual-22-12').embeddings[0]
}
query_start_time = time.time()

result = client.query.get(
        class_obj["class"], ["text"]
    ).with_near_vector(
        nearVector
    ).with_limit(2).with_additional(['certainty']).do()


print(" For 1 query, time taken for search = {0} ".format(time.time() - query_start_time))

print(json.dumps(result, indent=4))